In [3]:
import string 
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import matplotlib.pyplot as plt

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model

# small library for seeing the progress of loops.
#from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
tqdm().pandas()

0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/tqdm/std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
imagePath = '/Users/anjalisingh/Documents/ImageCaptionGenerator/Flicker8k_Dataset'

In [5]:
def extract_features(directory):
    model = VGG16()
    features = {}
    for img in tqdm(os.listdir(directory)):   # listing directories in images folder
        filename = directory + "/" + img
        image = Image.open(filename)
        image = image.resize((299, 299))
            # the image is now in an array of shape (3, 229, 229) 
            # need to expand it to (1, 3, 229, 229) as it's expecting a list
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0

        feature = model.predict(image)
        features[img] = feature
    return features

# 2048 feature vector
# features  = extract_features(imagePath)
# dump(features, open("features_VGG.p","wb"))    # features.p has been create which have image name and it's features.
# In a dictionary format image name is stored and its features as keys and every feature of a images are arranged in array


In [6]:
features = load(open("features_VGG.p", "rb"))

0it [00:00, ?it/s]


In [7]:
for img in os.listdir(imagePath):
    filename = imagePath + "/" + img
    #print(filename)

In [8]:
trainText = '/Users/anjalisingh/Documents/ImageCaptionGenerator/Description_of_images/Flickr_8k.trainImages.txt'

In [7]:
def load_doc(PathText):
    file = open(PathText, 'r')
    text = file.read()
    file.close()
    return text
# load the data
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos
train_imgs = load_photos(trainText)

In [8]:
def load_clean_descriptions(filename, photos):
    # loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words)<1:
            continue
        
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            
            desc = '<start>' + " ".join(image_caption) + '<end>'
            descriptions[image].append(desc)
            
    return descriptions

train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)

In [9]:
def load_features(photos):
    # loading all features
    all_features = load(open("features_VGG.p", "rb"))
    # selecting only needed features
    features = {k:all_features[k] for k in photos}
    return features

train_features = load_features(train_imgs)

In [10]:
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

In [11]:
def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    # fit_on_texts: Updates internal vocabulary based on a list of texts. This method creates the vocabulary 
    # index based on word frequency
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

tokenizer = create_tokenizer(train_descriptions)
# dump(tokenizer, open('tokenizer.p', 'wb'))
vocab_size = len(tokenizer.word_index) + 1
vocab_size

7577

In [12]:
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)
max_length = max_length(train_descriptions)
max_length

32

In [13]:
# X1, X2, y = list(), list(), list()
# # walk through each description for the image
# for key, desc_list in train_descriptions.items():
#     feature = features[key][0]
#         # encode the sequence
#     for cap in desc_list:
#         seq = tokenizer.texts_to_sequences([cap])[0]
#         # split one sequence into multiple X,y pairs
#         for i in range(1, len(seq)):
#             # split into input and output pair
#             in_seq, out_seq = seq[:i], seq[i]
#             # pad input sequence
#             in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
#             # encode output sequence
#             out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
#             # store
#             X1.append(feature)
#             X2.append(in_seq)
#             y.append(out_seq)

# X2 = np.array(X2)
# X1 = np.array(X1)
# y = np.array(y)
# X2.shape, X1.shape, y.shape

In [13]:
X1, X2, y = list(), list(), list()
for key, desc_list in train_descriptions.items():
    feature = features[key][0]
        # encode the sequence
    for cap in desc_list:
        seq = tokenizer.texts_to_sequences([cap])[0]
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=38)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)

In [ ]:
#Kernel died at this step
X2 = np.array(X2)
X1 = np.array(X1)
y = np.array(y)

In [14]:
def data_generator(descriptions, photos, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items():
            #retrieve photo features
            photo = photos[key][0]
            input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, description_list, photo)
            yield [[input_image, input_sequence], output_word]


def create_sequences(tokenizer, max_length, desc_list, photo):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            X1.append(photo)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

In [15]:
def define_model(vocab_size, max_length):
    
    # feature extractor model
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    # decoder model
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    # summarize model
#     print(model.summary())
#     plot_model(model, to_file='model.png', show_shapes=True)
    
    return model

In [16]:
model = define_model(vocab_size, max_length)
epochs = 20
steps = len(train_descriptions)
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('models/model_' + str(i) + '.h5')

ValueError: in user code:

    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:789 run_step  **
        outputs = model.t

In [ ]:
#!pip3 install tf-nightly-cpu-2.0-preview

In [ ]:
# features from the CNN model squeezed from 2048 to 256 nodes
inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
    
# LSTM sequence Model
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
    
# Merging both models
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
# tie it together [image, seq][word]
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
# model.layers[2].set_weights([emb_matrix]) 
# model.layers[2].trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam')
history = model.fit([X1, X2], y, epochs = 20, batch_size = 256) 
    
# summarize model
print(model.summary())

In [ ]:
model.save('models/IMCG_VGG.h5')